In [3]:
import csv
import numpy as np
from random import shuffle
import matplotlib.image as mpimg
import sklearn

#import cv2
#import matplotlib.pyplot as plt

samples = []

# Steering angle correction in order to add left and right pictures. 
# Maximum angle is 25 degrees so we want a float multiplying that number
# It seems the difference between side and center pictures is about 20 degrees
# that would imply a float multiplier of 0.8 for the side pictures if we wanted immediate correction
# However 0.8 seems too much so we'll start with lower than that
corrections = ['0.0', '0.2', '-0.2']

#count = 0

with open('data/cdriving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        center_info = [line[0], line[3], corrections[0]]
        left_info = [line[1], line[3], corrections[1]]
        right_info = [line[2], line[3], corrections[2]]
        samples.append(center_info)
        samples.append(left_info)
        samples.append(right_info)
    
print("samples is of length:",len(samples))

       
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.3)


def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            
            images = []
            angles = []
            for batch_sample in batch_samples:
                name = 'data/IMG/'+batch_sample[0].split('/')[-1]
                image = mpimg.imread(name)
                angle = float(batch_sample[1]) + float(batch_sample[2])
                images.append(image)
                angles.append(angle)
                
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)
                
                
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32) 

samples is of length: 41790


### Hello World

In [4]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from keras.layers import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(6,5,5, activation="relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(16,5,5, activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))
    
model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, samples_per_epoch= len(train_samples), 
                      validation_data=validation_generator, 
                      nb_val_samples=len(validation_samples), nb_epoch=3)

# model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=5)

model.save('model.h5')

Epoch 1/3
29253/29253 [==============================] - 482s - loss: 0.1001 - val_loss: 0.0183